# Stage-1 Multimodal Alignment Training

**Goal:** Train the connection layers (Adapters, Perceiver, Projector) to align frozen Vision (DINOv2) and Audio (Whisper) encoders with the Text Embedding space (Qwen).

**Architecture:**
1. **Frozen Encoders:** DINOv2 & Whisper
2. **Adapters:** Linear layers (Lazy initialization)
3. **Connector:** Perceiver Resampler (Learnable Latents)
4. **Projector:** MLP to LLM dimension
5. **Loss:** Matryoshka Contrastive Loss (MRL)


In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from functools import partial
import wandb
from dataclasses import asdict

# Local Imports
from imports.configs.config import setup_from_yaml
from imports.dataset import PixmoFeatureDataset, LibriSpeechFeatureDataset, collate_alignment
from imports.model import MultiModalAlignmentModel
from imports.align_training.text_encoder import HFTextEncoderConfig, HFTextEncoder
from imports.align_training.steps import AlignmentModules, AlignmentConfig
from imports.align_training.training import build_alignment_optimizer, train_alignment

/home/hice1/vchopra37/scratch/projects/edge_glass/edge_glass_env/lib/python3.12/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


## 1. Setup Configuration & Device

In [2]:
# Load configuration
cfg = setup_from_yaml("imports/configs/config.yaml")
device = cfg.torch_device
dtype = cfg.torch_dtype

print(f"\n🚀 Running on {device} with {dtype}")

wandb: Currently logged in as: vedaangchopra (vedaangchopra_gatech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[Config] Device: cuda, dtype: torch.float32
[Config] root_dir: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v1_code_base/edge_glass
[Config] features_dir: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v1_code_base/edge_glass/features

🚀 Running on cuda with torch.float32


## 2. Load Datasets (Vision & Audio)
We load both PixMo (Vision) and LibriSpeech (Audio) feature datasets.

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(cfg.models.llm_model_name)

# 1. Define Datasets
pixmo_train = PixmoFeatureDataset(cfg.datasets.pixmo_train_index)
pixmo_val   = PixmoFeatureDataset(cfg.datasets.pixmo_val_index)

# Add Audio Dataset
if cfg.datasets.librispeech_train_index:
    libri_train = LibriSpeechFeatureDataset(cfg.datasets.librispeech_train_index)
else:
    libri_train = [] # Empty if not configured
    print("⚠️ No Audio Index found in config.")

print(f"\n📚 Dataset Sizes:\n  - Vision Train: {len(pixmo_train)}\n  - Vision Val:   {len(pixmo_val)}\n  - Audio Train:  {len(libri_train)}")

# 2. Define Collate Function
collate_fn = partial(collate_alignment, tokenizer=tokenizer)

# 3. Create DataLoaders
vision_train_loader = DataLoader(
    pixmo_train, 
    batch_size=cfg.training.batch_size, 
    shuffle=True, 
    num_workers=4, 
    collate_fn=collate_fn
)

if len(libri_train) > 0:
    audio_train_loader = DataLoader(
        libri_train, 
        batch_size=cfg.training.batch_size, 
        shuffle=True, 
        num_workers=4, 
        collate_fn=collate_fn
    )
else:
    audio_train_loader = None

# Bundle them for the training loop
train_loaders = {"vision": vision_train_loader}
if audio_train_loader: train_loaders["audio"] = audio_train_loader

val_loaders = {"vision": DataLoader(pixmo_val, batch_size=cfg.training.batch_size, collate_fn=collate_fn)}

[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/train_feat_769.pt
[PixmoFeatureDataset] Loaded 872 valid entries from 873 total.
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_97.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_98.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_8.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_10.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_19.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_21.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_55.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_57.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: 

## 3. Initialize Frozen Text Encoder (Target)
This provides the "ground truth" embeddings we are trying to match.

In [4]:
txt_cfg = HFTextEncoderConfig(
    model_name=cfg.models.llm_model_name,
    max_length=128,
    trainable=False,  # Stage-1: Frozen
)

text_encoder = HFTextEncoder(cfg=txt_cfg, device=device, dtype=dtype)
d_text = text_encoder.hidden_size
print(f"✅ Text Encoder Loaded. Embedding Dim: {d_text}")

# Helper wrapper for training loop
def text_embed_fn(texts: list[str], max_length: int) -> torch.Tensor:
    with torch.no_grad():
        return text_encoder.encode(texts).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Text Encoder Loaded. Embedding Dim: 2048


## 4. Initialize Alignment Model & Warmup Adapters
**Crucial Step:** We must pass dummy data through the model once to initialize the "Lazy" Linear Adapters before we can build the optimizer.

In [5]:
model = MultiModalAlignmentModel(
    d_shared=512,
    d_latent=512,
    d_align=d_text,   # Must match Qwen dimension
    num_latents=cfg.architecture.num_latents,
    num_layers=cfg.architecture.num_perceiver_layers,
    num_heads=cfg.architecture.num_attn_heads,
    use_perceiver=True,
    dtype=torch.float32, # MRL is more stable in float32
    device=device,
)



In [6]:
print("⚡ Warming up to initialize lazy adapters...")
# Create dummy inputs on device
dummy_vision = torch.randn(2, 256, 1536).to(device)  # (B, T, D_vis)
dummy_audio  = torch.randn(2, 1000, 512).to(device)   # (B, T, D_aud)

# Force initialization
model._ensure_vision_adapter(1536) 
_ = model.vision_adapter(dummy_vision)

model._ensure_audio_adapter(512)
_ = model.audio_adapter(dummy_audio)

print("✅ Adapters initialized successfully.")

# Wrap modules for training
modules = AlignmentModules(
    vision_adapter=model.vision_adapter,
    audio_adapter=model.audio_adapter,
    perceiver=model.perceiver,
    projector=model.projector,
)

⚡ Warming up to initialize lazy adapters...
✅ Adapters initialized successfully.


## 5. Optimizer & W&B Setup

In [7]:
# Alignment Config for MRL Loss
align_cfg = AlignmentConfig(
    mrl_dims=tuple(cfg.mrl.mrl_dims) if cfg.mrl.mrl_dims else (d_text,),
    mrl_temperature=cfg.mrl.mrl_temp,
)

# Build Optimizer
optimizer = build_alignment_optimizer(
    modules=modules,
    learning_rate=float(cfg.training.learning_rate),
    weight_decay=float(cfg.training.weight_decay),
)
print(f"🔥 Optimizer ready. Params to train: {sum(p.numel() for p in optimizer.param_groups[0]['params'] if p.requires_grad)}")

# W&B Init
if cfg.misc.use_wandb:
    wandb.init(
        project=cfg.misc.wandb_project,
        name=cfg.misc.wandb_run_name,
        config=asdict(cfg),
    )
    def wb_log_fn(stats): wandb.log(stats)
else:
    wb_log_fn = None

🔥 Optimizer ready. Params to train: 21574656


## 6. Run Training Loop
This loop will alternate between Vision and Audio batches for each epoch.

In [9]:
# Determine available modalities
active_modalities = ["vision"]
if audio_train_loader:
    active_modalities.append("audio")

train_alignment(
    train_loaders=train_loaders,
    val_loaders=val_loaders,
    modules=modules,
    cfg=align_cfg,
    text_embed_fn=text_embed_fn,
    optimizer=optimizer,
    device=device,
    num_epochs=cfg.training.num_epochs,
    log_every=cfg.training.log_every_steps,
    log_fn=wb_log_fn,
    modalities=tuple(active_modalities),
)

[DEBUG] h_mod norm mean/std: 7.969107627868652 0.12253575026988983 | h_txt norm mean/std: 173.0849609375 18.46995735168457
[DEBUG] h_mod norm mean/std: 9.734088897705078 0.20857372879981995 | h_txt norm mean/std: 177.25059509277344 12.849637985229492
[DEBUG] h_mod norm mean/std: 11.058965682983398 0.15246635675430298 | h_txt norm mean/std: 176.8023223876953 14.764145851135254
[DEBUG] h_mod norm mean/std: 11.86959457397461 0.11283068358898163 | h_txt norm mean/std: 180.7265625 12.398889541625977
[DEBUG] h_mod norm mean/std: 12.934972763061523 0.1258547008037567 | h_txt norm mean/std: 180.41448974609375 11.174077033996582
[DEBUG] h_mod norm mean/std: 13.833272933959961 0.13960060477256775 | h_txt norm mean/std: 179.71572875976562 12.692340850830078
[DEBUG] h_mod norm mean/std: 14.824313163757324 0.24101030826568604 | h_txt norm mean/std: 171.9484100341797 19.055761337280273
[DEBUG] h_mod norm mean/std: 15.618216514587402 0.18838518857955933 | h_txt norm mean/std: 179.56881713867188 16.05

## 7. Inference Preview (Connecting to Qwen)
Here we test if the aligned features can actually be fed into the LLM as a soft prompt.

In [12]:
from transformers import AutoModelForCausalLM

@torch.no_grad()
def test_generation(image_path_or_feat, prompt="Describe this image."):
    print("\n--- Inference Test ---")
    
    # 1. Get Aligned Features (Soft Prompt)
    # Note: For this demo, we assume image_path_or_feat is a preprocessed tensor from our dataset
    # In real deployment, you'd use model.vision_encoder.encode_images(PIL_Image)
    
    # Simulating an input from the dataset
    feats = image_path_or_feat.unsqueeze(0).to(device) # (1, T, D)
    feats = feats.to(dtype=dtype)
    mask = torch.ones(1, feats.shape[1]).bool().to(device)
    
    # Pass through our trained connector
    tokens = modules.vision_adapter(feats)
    latents = modules.perceiver(tokens, encoder_mask=mask)
    soft_prompt = modules.projector(latents) # (1, L, d_text)
    
    print(f"Soft Prompt Shape: {soft_prompt.shape}")
    
    # 2. Load Decoder (Qwen)
    llm = AutoModelForCausalLM.from_pretrained(
        cfg.models.llm_model_name, 
        torch_dtype=dtype, 
        device_map="auto"
    )
    
    # 3. Embed Text Prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(llm.device)
    txt_embeds = llm.model.embed_tokens(inputs.input_ids)
    
    # 4. Concatenate: [Soft_Prompt, Text_Prompt]
    # Ensure soft_prompt matches LLM dtype
    soft_prompt = soft_prompt.to(dtype=llm.dtype, device=llm.device)
    
    inputs_embeds = torch.cat([soft_prompt, txt_embeds], dim=1)
    
    # 5. Generate
    out = llm.generate(inputs_embeds=inputs_embeds, max_new_tokens=30)
    generated_text = tokenizer.decode(out[0], skip_special_tokens=True)
    
    print(f"Prompt: {prompt}")
    print(f"Generated: {generated_text}")

In [ ]:

# Run test on a random sample from train set
sample_feat = pixmo_train[0]["features"]
test_generation(sample_feat)